# Pràctica 1 - El procés de l'aprenentatge automàtic {-}

**Autors:**  Marc Melia Flexas y Martin Mitrovski Delov

## Taula de continguts {-}

 1. [Introducció](#introduccio)
 2. [Tractament de dades](#tractament)
 3. [Experiments realitzats](#experiments)
 4. [Anàlisi dels resultats](#analisi)

# Introducció {#introduccio}

El projecte...jhkjhkj

El conjunt de dades que utilitzarem és Fashion Mnist

**Fashion Mnist**

Aquest conjunt de dades conté 70.000 imatges en escala de grisos en 10 categories. Les imatges mostren peces de roba individuals a baixa resolució (28 per 28 píxels). Fashion MNIST està pensat com una evolució directa del conjunt de dades clàssic MNIST que conté digits escrit a mà. Utilitzarem Fashion MNIST perquè presenta un problema una mica més difícil que el MNIST normal. Tots dos conjunts de dades són relativament petits i s'utilitzen per verificar que un algorisme funciona com s'esperava. Són bons punts de partida per provar i depurar codi, el que es coneix com a base line. 

## Libreries utilitzades

In [9]:
import pandas as pd
import numpy as np
import time

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from skimage import feature
from sklearn.model_selection import cross_val_score, KFold

from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA

# Tractament de dades {#tractament}

In [2]:
# Carregam el conjunt de dades
train = pd.read_csv('./dat/fashion-mnist_train.csv')
test = pd.read_csv('./dat/fashion-mnist_test.csv')

# Separam el conjunt d'entrenament en característiques X_train i etiquetes Y_train
X_train= train.drop(['label'],axis = 1)
y_train = train['label']

# Separam el conjunt de test en característiques X_train i etiquetes Y_train
X_test= test.drop(['label'],axis = 1)
y_test = test['label']

# Dimensió del conjunt de característiques
print(X_train.shape)

# Dimensió de les etiquetes
print(y_train.shape)

print(X_test.shape)

print(y_test.shape)

#X_train = X_train.astype('float32')
#X_val = X_val.astype('float32')
#X_test = X_test.astype('float32')

#Normalizar los valores de píxeles dividiéndolos por 255.0
#X_train /= 255.0
#X_val /= 255.0
#X_test /= 255.0

print(X_train)

(60000, 784)
(60000,)
(10000, 784)
(10000,)
       pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  pixel9  \
0           0       0       0       0       0       0       0       0       0   
1           0       0       0       0       0       0       0       0       0   
2           0       0       0       0       0       0       0       5       0   
3           0       0       0       1       2       0       0       0       0   
4           0       0       0       0       0       0       0       0       0   
...       ...     ...     ...     ...     ...     ...     ...     ...     ...   
59995       0       0       0       0       0       0       0       0       0   
59996       0       0       0       0       0       0       0       0       0   
59997       0       0       0       0       0       0       0       0       0   
59998       0       0       0       0       0       0       0       0       0   
59999       0       0       0       0       0       0       0    

In [31]:
# Función para aplicar LBP a una imagen
def apply_lbp(image):
    # Ajustar la forma de la imagen a (28, 28) si es necesario
    image = np.array(image).reshape(28, 28)
    

    # Aplicar LBP a la imagen
    lbp_image = feature.local_binary_pattern(image, P=24, R=3)

    # Aplanar la imagen LBP
    lbp_image_flat = lbp_image.flatten()
    
    #print(lbp_image_flat)

    return lbp_image_flat

# Aplicar LBP a todas las imágenes de entrenamiento y prueba
X_train_lbp = np.array([apply_lbp(image) for image in X_train.values])
X_test_lbp = np.array([apply_lbp(image) for image in X_test.values])

# Verificar las nuevas formas
print("Forma original de X_train:", X_train.shape)
print("Forma de X_train después de aplicar LBP:", X_train_lbp.shape)

#X_train_lbp /= 255.0
#X_val /= 255.0
#X_test_lbp /= 255.0

print(X_train_lbp)

print(X_train_lbp[0][0])


# Ahora puedes utilizar X_train_lbp y X_test_lbp en lugar de X_train y X_test en tu modelo SVM


Forma original de X_train: (60000, 784)
Forma de X_train después de aplicar LBP: (60000, 784)
[[16777215. 16777215. 16777215. ... 16777215. 16777215. 16777215.]
 [16777215. 16777215. 16777215. ... 16777215. 16777215. 16777215.]
 [16777215. 16777215. 16777215. ... 16777215. 16777215. 16777215.]
 ...
 [16777215. 16777215. 16777215. ... 16777215. 16777215. 16777215.]
 [16777215. 16777215. 16777215. ... 16777215. 16777215. 16777215.]
 [16777215. 16777215. 16777215. ... 16777215. 16777215. 16777215.]]
16777215.0


In [3]:
def apply_hog(image):
    # Ajustar la forma de la imagen a (28, 28) si es necesario
    image = np.array(image).reshape(28, 28)

    # Calcular las características HOG
    hog_features, hog_image = feature.hog(image, orientations=8, pixels_per_cell=(4, 4),
                                          cells_per_block=(1, 1), visualize=True)

    # Aplanar las características HOG
    hog_features_flat = hog_features.flatten()

    return hog_features_flat


X_train_hog = np.array([apply_hog(image) for image in X_train.values])
X_test_hog = np.array([apply_hog(image) for image in X_test.values])

# Verificar las nuevas formas
print("Forma original de X_train:", X_train.shape)
print("Forma de X_train después de aplicar LBP:", X_train_hog.shape)


Forma original de X_train: (60000, 784)
Forma de X_train después de aplicar LBP: (60000, 392)


In [5]:
# Normalizar StandardScaler()
scaler = StandardScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

#PCA
# Especificar el número de componentes principales que deseas retener
n_components = 0.85  # Ajusta según tus necesidades

# Inicializar PCA
pca = PCA(n_components=n_components)

# Ajustar y transformar los datos de entrenamiento
X_train_pca = pca.fit_transform(X_train_normalized)

# Transformar los datos de prueba (usando los mismos componentes principales aprendidos del conjunto de entrenamiento)
X_test_pca = pca.transform(X_test_normalized)

print(X_train_pca.shape)
print(X_test_pca.shape)

print(X_train_pca)
print(X_test_pca)

(60000, 81)
(10000, 81)
[[ 1.06649729e+01  1.49933635e+01 -6.89468090e-01 ...  2.30535372e-01
   9.81395644e-01 -8.84122042e-01]
 [-1.19897476e+01  1.18127701e+01 -5.80104873e+00 ...  8.67317161e-01
  -1.22617381e+00 -5.91727962e-01]
 [ 2.05176712e+01  1.57978434e+00  6.77012222e+00 ... -1.16864241e-01
  -1.48010090e-02 -4.76387998e-01]
 ...
 [ 7.14876699e+00 -5.43564927e-01 -8.74059571e-01 ... -6.37226067e-01
  -4.69789395e-01  8.86471004e-02]
 [ 4.43415256e+00  2.08327043e+01 -5.76302113e-01 ...  2.95631752e-01
  -2.01054401e+00 -1.37214034e+00]
 [-9.10680985e+00  1.48311538e+01 -3.67467806e+00 ... -2.03501184e+00
  -1.84937814e+00  7.12581912e-01]]
[[ 15.98149283  -4.59762206  -1.12973407 ...   1.05773224   1.78713826
    0.61097464]
 [ -1.32965543 -17.28430805  -6.21186681 ...  -0.25380406  -0.50499781
   -0.34996485]
 [ -9.77009012  -3.29941672  11.43858573 ...  -0.31222027   0.53711554
   -1.06017376]
 ...
 [  4.5119519   10.03817628   4.8921672  ...  -0.33350819  -0.5149351
   -

# Experiments realitzats {#experiments}

## Experiment 1

En el primer experiment que volem realizar volem veure quin és el resultat d'utilitzar un SVM amb els paràmetres per defecte, sense modificarne cap. Els paràmetres per defecte de la classe SVC de la llibreria sklear son els seguents:

SVC(C=1.0, kernel='rbf', degree=3, gamma='scale', coef0=0.0, shrinking=True, probability=False, tol=1e-3, cache_size=200, class_weight=None, verbose=False, max_iter=-1, decision_function_shape='ovr', break_ties=False, random_state=None)

-sense normalitzar

In [3]:
# Iniciar el temporizador antes de entrenar el modelo
start_time = time.time()

# Inicializar el clasificador SVM
svm_classifier = SVC()

# Entrenar el modelo SVM
svm_classifier.fit(X_train, y_train)

# Detener el temporizador después de entrenar el modelo
end_time = time.time()

# Calcular el tiempo transcurrido
elapsed_time = end_time - start_time
print(f"Tiempo de entrenamiento: {elapsed_time} segundos")

y_test_pred = svm_classifier.predict(X_test)
    
# Evaluar el rendimiento en el conjunto de test
accuracy = accuracy_score(y_test, y_test_pred)
print(f"Accuracy on test set: {accuracy}")

Tiempo de entrenamiento: 248.13191533088684 segundos
Accuracy on test set: 0.8921


In [ ]:
    clf = SVC(kernel='linear', C=1)

    # Configura k-fold cross-validation
    k_fold = KFold(n_splits=3, shuffle=True, random_state=42)

    # Realiza k-fold cross-validation y obtén las puntuaciones de precisión
    cross_val_scores = cross_val_score(clf, X_train, y_train, cv=k_fold, scoring='accuracy')

    # Imprime las puntuaciones de precisión para cada fold
    print("Puntuaciones de precisión para cada fold:", cross_val_scores)

    # Imprime la precisión promedio
    print("Precisión promedio:", np.mean(cross_val_scores))

Explicar resultats experiment 1

## Experiment 2

GridSearch

KFold 

In [ ]:
# Definir los parámetros y los valores a probar
param_grid = {'C': [0.1, 1, 10, 100], 
              'kernel': ['linear', 'rbf', 'poly','sigmoid'], 
              'gamma': ['scale', 'auto']}

# Inicializar el clasificador  SVM
svm_classifier = SVC()  

# Inicializar GridSearchCV con el clasificador y los parámetros a probar
grid_search = GridSearchCV(estimator=svm_classifier, param_grid=param_grid, cv=5)

# Iniciar el temporizador antes de entrenar el modelo
start_time = time.time()

# Realizar la búsqueda de cuadrícula en los datos
grid_search.fit(X_train_pca, y_train)

# Detener el temporizador después de entrenar el modelo
end_time = time.time()

# Calcular el tiempo transcurrido
elapsed_time = end_time - start_time
print(f"Tiempo de entrenamiento: {elapsed_time} segundos")

# Imprimir los mejores parámetros y el mejor rendimiento
print("Mejores parámetros:", grid_search.best_params_)
print("Mejor rendimiento (accuracy):", grid_search.best_score_)

# Obtener el mejor clasificador entrenado
best_classifier = grid_search.best_estimator_

# Realizar predicciones en el conjunto de prueba
y_pred = best_classifier.predict(X_test_pca)

# Evaluar el rendimiento en el conjunto de test
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy on test set: {accuracy}")

In [ ]:
# Definir los parámetros y los valores a probar
param_grid = {'C': [0.1, 1], 
              'kernel': ['linear', 'rbf'], 
              'gamma': [0.1, 1]}

# Inicializar el clasificador  SVM
svm_classifier = SVC()  

# Inicializar GridSearchCV con el clasificador y los parámetros a probar
grid_search = GridSearchCV(estimator=svm_classifier, param_grid=param_grid, cv=5, n_jobs=-1)

# Iniciar el temporizador antes de entrenar el modelo
start_time = time.time()

# Realizar la búsqueda de cuadrícula en los datos
grid_search.fit(X_train_pca, y_train)

# Detener el temporizador después de entrenar el modelo
end_time = time.time()

# Calcular el tiempo transcurrido
elapsed_time = end_time - start_time
print(f"Tiempo de entrenamiento: {elapsed_time} segundos")

# Imprimir los mejores parámetros y el mejor rendimiento
print("Mejores parámetros:", grid_search.best_params_)
print("Mejor rendimiento (accuracy):", grid_search.best_score_)

# Obtener el mejor clasificador entrenado
best_classifier = grid_search.best_estimator_

# Realizar predicciones en el conjunto de prueba
y_pred = best_classifier.predict(X_test_pca)

# Evaluar el rendimiento en el conjunto de test
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy on test set: {accuracy}")

## Experiment 3

PCA

## Experiment 4

HOP

# Anàlisi dels resultats {#analisi}

asdasdasdasd m,